In [5]:
import numpy as np
import scipy.sparse as spa
import scipy.sparse.linalg as las
import time as ti

def fun_f_a(x, n):
    f = np.sum([100 * (x[i+1] - x[i]**2)**2 + (x[i] - 1)**2 for i in range(0, n-1)])
    return f

def fun_g_a(x, n):
    g = np.zeros(n)
    g[1:] = -200 * np.power(x[:-1], 2)
    g[:-1] += -400 * x[:-1] * (-np.power(x[:-1], 2) + x[1:])
    g[1:-1] += 202 * x[1:-1]
    g[0] += 2 * x[0]
    g[-1] += 200 * x[-1]
    g[:-1] += -2
    return g

def fun_H_a(x, n):
    ds = np.array([-400 * x[i] for i in range(0, n-1)])
    dp = np.zeros(n)
    dp[:-1] = np.array([1200 * x[i]**2 - 400 * x[i+1] for i in range(0, n-1)])
    dp[-1] = 200
    dp[1:-1] += 202
    dp[0] += 2
    ind = np.arange(0, n)
    I = np.concatenate((ind, ind[:-1], ind[:-1] + 1))
    J = np.concatenate((ind, ind[:-1] + 1, ind[:-1]))
    V = np.concatenate((dp, ds, ds))
    H = spa.coo_matrix((V, (I, J)))
    return H

def alpha_estimate_spa(xk, d_k, M):
    a = 0; b = 1; n = len(xk)
    for k in range(0, M):
        alp = np.linspace(a, b, num=5, endpoint=True)
        alp1 = alp[1]
        alp2 = alp[2]
        alp3 = alp[3]
        xk1 = xk + alp1 * d_k
        xk2 = xk + alp2 * d_k
        xk3 = xk + alp3 * d_k
        f1 = fun_f_a(xk1, n)
        f2 = fun_f_a(xk2, n)
        f3 = fun_f_a(xk3, n)
        pos = np.argmin([f1, f2, f3]) + 1
        a = alp[pos - 1]
        b = alp[pos + 1]
    return (a + b) / 2

def finite_diff_grad(f, x, h=1e-5):
    n = len(x)
    grad = np.zeros(n)
    fx = f(x, n)  # make sure to pass 'n' as parameter
    for i in range(n):
        x[i] += h
        grad[i] = (f(x, n) - fx) / h  # make sure to pass 'n' as parameter
        x[i] -= h
    return grad

def finite_diff_hess(f, x, h=1e-5):
    n = len(x)
    hessian = np.zeros((n, n))
    for i in range(n):
        x[i] += h; fxi = f(x, n)  # make sure to pass 'n' as parameter
        for j in range(i, n):
            x[j] += h; fxij = f(x, n)  # make sure to pass 'n' as parameter
            if i == j:
                hessian[i, j] = (fxij - 2*fxi + f(x-h, n)) / h**2  # make sure to pass 'n' as parameter
            else:
                x[i] -= h; fxj = f(x, n)  # make sure to pass 'n' as parameter
                hessian[i, j] = hessian[j, i] = (fxij - fxi - fxj + f(x-h, n)) / (2*h**2)  # make sure to pass 'n' as parameter
            x[j] -= h
        x[i] -= h
    return hessian

def newton_method(f, grad, hess, x0, alpha_search, tol=1e-6, max_iter=50):
    xk = x0.copy()
    norm_gk = []
    for _ in range(max_iter):
        gk = grad(f, xk)
        Hk = hess(f, xk)
        norm_gk.append(np.linalg.norm(gk))
        dxk = np.linalg.solve(Hk, -gk)
        alpha = alpha_search(xk, dxk, 20)  # Added max iterations for alpha search
        xk += alpha * dxk
        if np.linalg.norm(gk) < tol:
            break
    return xk, norm_gk

n = 10**6
x0 = 5 * np.ones(n)

# Método de Newton con cálculos exactos
start_time = ti.time()
x_min_exact, norms_exact = newton_method(fun_f_a, fun_g_a, fun_H_a, x0, alpha_estimate_spa, tol=1e-6, max_iter=50)
exact_time = ti.time() - start_time

# Método de Newton con diferencias finitas
start_time = ti.time()
x_min_approx, norms_approx = newton_method(fun_f_a, finite_diff_grad, finite_diff_hess, x0, alpha_estimate_spa, tol=1e-6, max_iter=50)
approx_time = ti.time() - start_time

# Resultados y comparación
print("Tiempo con cálculos exactos: ", exact_time)
print("Norma del gradiente con cálculos exactos: ", norms_exact[-1])
print("Tiempo con diferencias finitas: ", approx_time)
print("Norma del gradiente con diferencias finitas: ", norms_approx[-1])
print("Diferencia en soluciones: ", np.linalg.norm(x_min_exact - x_min_approx))


ValueError: maximum supported dimension for an ndarray is 32, found 1000000